**PM5: Decision Trees**

Team #4

Dataset: [Solar Power Generation](https://www.kaggle.com/anikannal/solar-power-generation-datas)

# Imports

In [1]:
# Imports
import plotly.express as px
import numpy as np
import pandas as pd
import datetime
#import time #PH - add if the code below requires it

# Prepping Dataset

## Loading Data

In [2]:
#https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
url = 'https://github.com/phestvik/Amii_SolarPowerGenerationData/blob/main/'
raw_flag = '?raw=true'

file1 = url + 'Plant_1_Generation_Data.csv' + raw_flag
file2 = url + 'Plant_1_Weather_Sensor_Data.csv' + raw_flag
file3 = url + 'Plant_2_Generation_Data.csv' + raw_flag
file4 = url + 'Plant_2_Weather_Sensor_Data.csv' + raw_flag

df_plant1_gen = pd.read_csv(file1)
df_plant1_sensor = pd.read_csv(file2)
df_plant2_gen = pd.read_csv(file3)
df_plant2_sensor = pd.read_csv(file4)

## Transformations

**Rename Columns**

In [3]:
df_plant1_sensor['SOURCE_KEY'].nunique()

1

In [4]:
df_plant1_sensor['PLANT_ID'].nunique()

1

In [5]:
#Renaming Source Key Column into its true descriptor for generation data
df_plant1_gen.rename(columns={'SOURCE_KEY':'INVERTER_ID'}, inplace=True)
df_plant2_gen.rename(columns={'SOURCE_KEY':'INVERTER_ID'}, inplace=True)
#Dropping Source Key Column from sensor data because it is the same for the entire file
df_plant1_sensor.drop(columns=['SOURCE_KEY'], inplace=True)
df_plant2_sensor.drop(columns=['SOURCE_KEY'], inplace=True)
#Dropping Plant ID from sensor data because it is the same for the entire file
df_plant1_sensor.drop(columns=['PLANT_ID'], inplace=True)
df_plant2_sensor.drop(columns=['PLANT_ID'], inplace=True)

 **DATE_TIME from object to datetime64**

In [6]:
#Passing errors=’coerce’ will force an out-of-bounds date to NaT, in addition to forcing non-dates (or non-parseable dates) to NaT.
df_plant1_gen["DATE_TIME"] = pd.to_datetime(df_plant1_gen["DATE_TIME"], errors='coerce', format='%d-%m-%Y %H:%M')
df_plant1_sensor["DATE_TIME"] = pd.to_datetime(df_plant1_sensor["DATE_TIME"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_plant2_gen["DATE_TIME"] = pd.to_datetime(df_plant2_gen["DATE_TIME"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_plant2_sensor["DATE_TIME"] = pd.to_datetime(df_plant2_sensor["DATE_TIME"], errors='coerce', format='%Y-%m-%d %H:%M:%S')

**Scaling Plant 1 Power Values**

In [7]:
df_plant1_gen.sample(10)

,DATE_TIME,PLANT_ID,INVERTER_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
4238,2020-05-17 03:45:00,4135001,zBIq5rxdHJRwDNY,0.000000,0.000000,0.000000,6352150.000
58418,2020-06-13 01:45:00,4135001,3PZuoBAID5Wc2HD,0.000000,0.000000,0.000000,7202086.000
49528,2020-06-08 20:45:00,4135001,1BY6WEcLGh8j5v7,0.000000,0.000000,7864.000000,6431614.000
41571,2020-06-05 01:15:00,4135001,7JYdWkrLSPkdwr4,0.000000,0.000000,0.000000,7756621.000
41484,2020-06-05 00:15:00,4135001,McdE0feGgRqW7Ca,0.000000,0.000000,0.000000,7316761.000
59330,2020-06-13 12:00:00,4135001,iCRJl6heRkivqQ3,13521.857140,1318.357143,4140.000000,7395499.000
26866,2020-05-28 16:15:00,4135001,iCRJl6heRkivqQ3,4114.625000,403.475000,7876.375000,7282568.375
29440,2020-05-30 06:00:00,4135001,wCURE6d3bPkepu2,81.625000,7.887500,0.000000,6895880.000
29525,2020-05-30 07:00:00,4135001,rGa61gmuvPhdLxV,1696.571429,166.028571,72.571429,7223146.571
43506,2020-06-05 23:30:00,4135001,iCRJl6heRkivqQ3,0.000000,0.000000,7803.000000,7341803.000


In [8]:
df_plant2_gen.sample(10)

,DATE_TIME,PLANT_ID,INVERTER_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
1305,2020-05-15 14:45:00,4136001,Mx2yZCDsyf6DPfv,1007.706667,985.246667,6178.2,2.459824e+06
22144,2020-05-26 21:30:00,4136001,4UPUqMRk7TRMgml,0.000000,0.000000,9501.0,2.516326e+06
10178,2020-05-19 20:00:00,4136001,mqwcsP2rE7J0TFp,0.000000,0.000000,6683.0,5.936187e+08
16452,2020-05-23 14:15:00,4136001,q49J1IKaHRwDQnt,0.000000,0.000000,1555.0,3.850590e+05
50862,2020-06-10 00:30:00,4136001,oZZkBaNadn6DNKz,0.000000,0.000000,0.0,1.708251e+09
39381,2020-06-04 14:00:00,4136001,vOuJvMaM2sgwLmb,393.426667,386.413333,5298.6,2.351522e+06
55097,2020-06-12 00:45:00,4136001,LYwnQax7tkwH5Cb,0.000000,0.000000,3718.0,1.795083e+09
8075,2020-05-18 20:00:00,4136001,81aHJ1q11NBPMrL,0.000000,0.000000,3485.0,1.215296e+09
33026,2020-06-01 13:30:00,4136001,4UPUqMRk7TRMgml,47.400000,45.800000,4014.0,2.556573e+06
17103,2020-05-23 23:15:00,4136001,xoJJ8DcxJEcupym,0.000000,0.000000,9380.0,2.091867e+08


For Plant 1, DC_POWER appears to be 10x what is should be

In [9]:
df_plant1_gen['DC_POWER'] = df_plant1_gen['DC_POWER'] / 10

## Merge Plant and Sensor

**Merge Plant 1 and Sensor 1 | Plant 2 and Sensor 2**

In [10]:
display(df_plant1_gen)
display(df_plant1_sensor)

,DATE_TIME,PLANT_ID,INVERTER_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.000,6259559.0
1,2020-05-15 00:00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.000,6183645.0
2,2020-05-15 00:00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.000,6987759.0
3,2020-05-15 00:00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.000,7602960.0
4,2020-05-15 00:00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.000,7158964.0
...,...,...,...,...,...,...,...
68773,2020-06-17 23:45:00,4135001,uHbuxQJl8lW7ozc,0.0,0.0,5967.000,7287002.0
68774,2020-06-17 23:45:00,4135001,wCURE6d3bPkepu2,0.0,0.0,5147.625,7028601.0
68775,2020-06-17 23:45:00,4135001,z9Y9gH1T5YWrNuG,0.0,0.0,5819.000,7251204.0
68776,2020-06-17 23:45:00,4135001,zBIq5rxdHJRwDNY,0.0,0.0,5817.000,6583369.0


,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,24.621525,22.165423,0.0
...,...,...,...,...
3177,2020-06-17 22:45:00,22.150570,21.480377,0.0
3178,2020-06-17 23:00:00,22.129816,21.389024,0.0
3179,2020-06-17 23:15:00,22.008275,20.709211,0.0
3180,2020-06-17 23:30:00,21.969495,20.734963,0.0


In [11]:
dfplant1 = df_plant1_gen.merge(df_plant1_sensor, how='inner', left_on= 'DATE_TIME', right_on= 'DATE_TIME', sort=True)
dfplant2 = df_plant2_gen.merge(df_plant2_sensor, how='inner', left_on= 'DATE_TIME', right_on= 'DATE_TIME', sort=True)

In [12]:
dfplant1

,DATE_TIME,PLANT_ID,INVERTER_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.000,6259559.0,25.184316,22.857507,0.0
1,2020-05-15 00:00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.000,6183645.0,25.184316,22.857507,0.0
2,2020-05-15 00:00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.000,6987759.0,25.184316,22.857507,0.0
3,2020-05-15 00:00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.000,7602960.0,25.184316,22.857507,0.0
4,2020-05-15 00:00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.000,7158964.0,25.184316,22.857507,0.0
...,...,...,...,...,...,...,...,...,...,...
68769,2020-06-17 23:45:00,4135001,uHbuxQJl8lW7ozc,0.0,0.0,5967.000,7287002.0,21.909288,20.427972,0.0
68770,2020-06-17 23:45:00,4135001,wCURE6d3bPkepu2,0.0,0.0,5147.625,7028601.0,21.909288,20.427972,0.0
68771,2020-06-17 23:45:00,4135001,z9Y9gH1T5YWrNuG,0.0,0.0,5819.000,7251204.0,21.909288,20.427972,0.0
68772,2020-06-17 23:45:00,4135001,zBIq5rxdHJRwDNY,0.0,0.0,5817.000,6583369.0,21.909288,20.427972,0.0


# Feature Engineering

## Hour

Create HOUR from DATE_TIME

In [13]:
# Create time from date time
dfplant1['HOUR'] = dfplant1['DATE_TIME'].dt.hour
dfplant1['HOUR'] = dfplant1['HOUR'].astype('int')
#dfplant1['DATE'] = dfplant1['DATE_TIME'].dt.date #PH - commented out. not sure if we need this for the analysis

dfplant2['HOUR'] = dfplant2['DATE_TIME'].dt.hour
dfplant2['HOUR'] = dfplant2['HOUR'].astype('int')
#dfplant2['DATE'] = dfplant2['DATE_TIME'].dt.date #PH - commented out. not sure if we need this for the analysis

In [14]:
dfplant1

,DATE_TIME,PLANT_ID,INVERTER_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,HOUR
0,2020-05-15 00:00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.000,6259559.0,25.184316,22.857507,0.0,0
1,2020-05-15 00:00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.000,6183645.0,25.184316,22.857507,0.0,0
2,2020-05-15 00:00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.000,6987759.0,25.184316,22.857507,0.0,0
3,2020-05-15 00:00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.000,7602960.0,25.184316,22.857507,0.0,0
4,2020-05-15 00:00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.000,7158964.0,25.184316,22.857507,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
68769,2020-06-17 23:45:00,4135001,uHbuxQJl8lW7ozc,0.0,0.0,5967.000,7287002.0,21.909288,20.427972,0.0,23
68770,2020-06-17 23:45:00,4135001,wCURE6d3bPkepu2,0.0,0.0,5147.625,7028601.0,21.909288,20.427972,0.0,23
68771,2020-06-17 23:45:00,4135001,z9Y9gH1T5YWrNuG,0.0,0.0,5819.000,7251204.0,21.909288,20.427972,0.0,23
68772,2020-06-17 23:45:00,4135001,zBIq5rxdHJRwDNY,0.0,0.0,5817.000,6583369.0,21.909288,20.427972,0.0,23


## Inverter Loading Ratio (ILR)

In [15]:
dfplant1['IRL'] = dfplant1['DC_POWER'] / dfplant1['AC_POWER']
dfplant2['IRL'] = dfplant2['DC_POWER'] / dfplant2['AC_POWER']

## Plant Power

Calculate the the DC Power at 12pm (peak power) for the entire plant.
Remember DC_POWER is in KW

In [16]:
#dfplant1.query(f'(DATE_TIME > "{datetime.date(2020,5,15)}") and (DATE_TIME < "{datetime.date(2020,5,16)}")')
dfplant1.query(f'(DATE_TIME == "{datetime.datetime(2020,5,15,12,0,0)}")').sum()

PLANT_ID                                                        90970022
INVERTER_ID            1BY6WEcLGh8j5v71IF53ai7Xc0U56Y3PZuoBAID5Wc2HD7...
DC_POWER                                                         15582.2
AC_POWER                                                         15250.8
DAILY_YIELD                                                      59620.9
TOTAL_YIELD                                                  1.50821e+08
AMBIENT_TEMPERATURE                                              688.718
MODULE_TEMPERATURE                                               1082.04
IRRADIATION                                                      11.3292
HOUR                                                                 264
IRL                                                              22.4766
dtype: object

So Plant 1's peak power is ~156MW.

In [17]:
dfplant2.query(f'(DATE_TIME == "{datetime.datetime(2020,5,15,12,0,0)}")').DC_POWER.sum()

16969.051904761905

So Plant 2's peak power is ~17MW.

Check that MODULE_TEMPERATUERE AND IRRADIATION is the same for all INVERTER_IRs

In [18]:
dfplant1.query(f'(DATE_TIME == "{datetime.datetime(2020,5,15,12,0,0)}")')

,DATE_TIME,PLANT_ID,INVERTER_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,HOUR,IRL
1036,2020-05-15 12:00:00,4135001,1BY6WEcLGh8j5v7,823.250000,805.187500,2382.875000,6261941.875,31.305375,49.183584,0.514963,12,1.022433
1037,2020-05-15 12:00:00,4135001,1IF53ai7Xc0U56Y,701.314286,686.114286,2752.428571,6186397.429,31.305375,49.183584,0.514963,12,1.022154
1038,2020-05-15 12:00:00,4135001,3PZuoBAID5Wc2HD,640.257143,626.871429,2676.285714,6990435.286,31.305375,49.183584,0.514963,12,1.021353
1039,2020-05-15 12:00:00,4135001,7JYdWkrLSPkdwr4,691.071429,676.542857,2612.857143,7605572.857,31.305375,49.183584,0.514963,12,1.021475
1040,2020-05-15 12:00:00,4135001,McdE0feGgRqW7Ca,620.171429,607.442857,2888.714286,7161852.714,31.305375,49.183584,0.514963,12,1.020954
1041,2020-05-15 12:00:00,4135001,VHMLBKoKgIrUVDU,705.085714,690.171429,2644.000000,7209052.000,31.305375,49.183584,0.514963,12,1.021610
1042,2020-05-15 12:00:00,4135001,WRmjgnKYAwPKWDb,695.142857,680.642857,2605.714286,7031278.714,31.305375,49.183584,0.514963,12,1.021303
1043,2020-05-15 12:00:00,4135001,YxYtjZvoooNbGkE,824.562500,806.375000,2745.625000,7182711.625,31.305375,49.183584,0.514963,12,1.022555
1044,2020-05-15 12:00:00,4135001,ZnxXDlPa8U1GXgE,799.642857,782.142857,2688.285714,6524860.286,31.305375,49.183584,0.514963,12,1.022374
1045,2020-05-15 12:00:00,4135001,ZoEaEvLYb1n2sOq,793.150000,775.775000,2701.000000,7100800.000,31.305375,49.183584,0.514963,12,1.022397


In [19]:
dfplant1.query(f'(DATE_TIME == "{datetime.datetime(2020,5,15,12,0,0)}")').MODULE_TEMPERATURE.mean()
dfplant1.query(f'(DATE_TIME == "{datetime.datetime(2020,5,15,12,0,0)}")').INVERTER_ID.count()

22

**Create a DF for Plant Power**

In [20]:
listData = []

uniqueDate_Time = dfplant1['DATE_TIME'].unique()
for i in uniqueDate_Time:
  #nned to add a feature for # of inverters for each time stamp
  #to show this after do a histo or line plot to show the concept
  a = dfplant1.query(f'(DATE_TIME == "{i}")').DC_POWER.sum()
  b = dfplant1.query(f'(DATE_TIME == "{i}")').INVERTER_ID.count()
  c = dfplant1.query(f'(DATE_TIME == "{i}")').AMBIENT_TEMPERATURE.mean()
  d = dfplant1.query(f'(DATE_TIME == "{i}")').MODULE_TEMPERATURE.mean()
  e = dfplant1.query(f'(DATE_TIME == "{i}")').IRRADIATION.mean()
  f = dfplant1.query(f'(DATE_TIME == "{i}")').HOUR.mean()
  listData.append([i, a, b, c, d, e, f])

In [21]:
listData2 = []

uniqueDate_Time = dfplant2['DATE_TIME'].unique()
for i in uniqueDate_Time:
  #nned to add a feature for # of inverters for each time stamp
  #to show this after do a histo or line plot to show the concept
  a = dfplant2.query(f'(DATE_TIME == "{i}")').DC_POWER.sum()
  b = dfplant2.query(f'(DATE_TIME == "{i}")').INVERTER_ID.count()
  c = dfplant2.query(f'(DATE_TIME == "{i}")').AMBIENT_TEMPERATURE.mean()
  d = dfplant2.query(f'(DATE_TIME == "{i}")').MODULE_TEMPERATURE.mean()
  e = dfplant2.query(f'(DATE_TIME == "{i}")').IRRADIATION.mean()
  f = dfplant2.query(f'(DATE_TIME == "{i}")').HOUR.mean()
  listData2.append([i, a, b, c, d, e, f])

In [22]:
cols = ['DATE_TIME', 'PLANT_POWER', 'NUM_INVERTERS', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'HOUR']
dfplant1_PP = pd.DataFrame(listData, columns=cols)
dfplant2_PP = pd.DataFrame(listData2, columns=cols)

In [23]:
dfplant1_PP['HOUR'] = dfplant1_PP['HOUR'].astype('int8')
dfplant1_PP['NUM_INVERTERS'] = dfplant1_PP['NUM_INVERTERS'].astype('int8')

In [24]:
dfplant2_PP['HOUR'] = dfplant2_PP['HOUR'].astype('int8')
dfplant2_PP['NUM_INVERTERS'] = dfplant2_PP['NUM_INVERTERS'].astype('int8')

# Append DFs

**Append Plant 1 and Plant 2**

In [25]:
dfplant1_PP['PLANT_ID'] = 1
dfplant2_PP['PLANT_ID'] = 2

In [26]:
df_plants = dfplant1_PP.append(dfplant2_PP, ignore_index=True)

In [27]:
df_plants['PLANT_ID'] = df_plants['PLANT_ID'].astype('category')

In [28]:
df_plants.sample(10)

,DATE_TIME,PLANT_POWER,NUM_INVERTERS,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,HOUR,PLANT_ID
4592,2020-05-29 23:30:00,0.000000,22,27.425483,26.055791,0.000000,23,2
1615,2020-06-01 21:45:00,0.000000,22,23.353536,20.933164,0.000000,21,1
6023,2020-06-13 21:45:00,0.000000,22,24.633419,23.544276,0.000000,21,2
1862,2020-06-04 11:45:00,23406.276785,22,28.901027,50.751308,0.819862,11,1
4285,2020-05-26 18:30:00,406.753333,18,33.429941,32.581822,0.018105,18,2
4581,2020-05-29 20:45:00,0.000000,22,31.234285,29.934426,0.000000,20,2
4008,2020-05-23 21:15:00,0.000000,18,29.375490,28.162750,0.000000,21,2
6333,2020-06-17 03:15:00,0.000000,22,22.963709,22.606509,0.000000,3,2
5680,2020-06-10 08:00:00,7794.735238,22,25.954460,30.214725,0.236895,8,2
4289,2020-05-26 19:30:00,0.000000,18,32.079919,30.424236,0.000000,19,2


# Normalization

In [29]:
X_cols_num = ['NUM_INVERTERS','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','HOUR']
X_cols_cat = ['PLANT_ID']

#y_cols = ['PLANT_POWER']
y_cols = ['PLANT_POWER_CAT']

## Min-Max Scaling

In [30]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()

In [31]:
X_plants_mms = df_plants.reindex(columns=(X_cols_cat + X_cols_num)) #select the features cols for ML with category first and numerical last
scaled_data = minmax_scaler.fit_transform(X_plants_mms[X_cols_num]) #scale the numeric fields
df_scaled = pd.DataFrame(scaled_data, columns=X_plants_mms[X_cols_num].columns) #create a df from scaled data
X_plants_mms[X_cols_num] = df_scaled
y_plants_mms = df_plants[y_cols]

KeyError: ignored

In [ ]:
X_plants_mms

In [ ]:
X_plants_mms.info()

## Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
z_score_scaler = StandardScaler()

In [ ]:
whose_code = "Alfred"
if whose_code == 'Alfred':
  print(whose_code)
  X_plants_std = df_plants[X_cols_num + X_cols_cat]
  X_plants_std_num = X_plants_std.iloc[:, :-1]
  X_plants_std.iloc[:, :-1] = z_score_scaler.fit_transform(X_plants_std_num)
  y_plants_std = df_plants[y_cols]
  X_plants_std
else:
  print(whose_code)
  X_plants_std = df_plants.reindex(columns=(X_cols_cat + X_cols_num)) #select the features cols for ML with category first and numerical last
  scaled_data = z_score_scaler.fit_transform(X_plants_std[X_cols_num]) #scale the numeric fields
  df_scaled = pd.DataFrame(scaled_data, columns=X_plants_std[X_cols_num].columns) #create a df from scaled data
  X_plants_std[X_cols_num] = df_scaled
  y_plants_std = df_plants[y_cols]

In [ ]:
X_plants_std

In [ ]:
X_plants_std